In [40]:
from timeit import default_timer

# Hash table using an array and modulo as hash function

Task: Create a function `hashtable(entry)` which realizes the following mapping

|Input|Output|
|-|-|
0xa582041|4
0xa592041|8
0xa5a2041|3
0xa582042|1
0xa592042|5
0xa5a2042|9
0xa582043|7
0xa592043|2

Function to check if the hash function produces no collisions (== perfect hash function)

In [1]:
def isPerfectHashfunction(hash_function, inputs):
    return len({hash_function(x) for x in inputs}) == len(inputs)

Find value to use for modulo

In [4]:
inputs = [0xa582041, 0xa592041, 0xa5a2041, 0xa582042, 0xa592042, 0xa5a2042, 0xa582043, 0xa592043]
outputs = [4, 8, 3, 1, 5, 9, 7, 2]
i = 8


while True:
    hash_func = lambda x: x % i
    if isPerfectHashfunction(hash_func, inputs):
        print(i)
        break
    i += 1

12


Build lookup table

In [28]:
def buildLookupTable(hash_func, inputs, outputs):
    lut = [0] * (max(hash_func(x) for x in inputs) + 1)
    for inp, out in zip(inputs, outputs):
        index = hash_func(inp)
        lut[index] = out
    return lut


lut = buildLookupTable(lambda x: x % 12, inputs, outputs)
lut

[0, 8, 5, 2, 0, 3, 9, 0, 0, 4, 1, 7]

Create the function

In [29]:
def hashtable(entry):
    return lut[entry % 12]

Test the implemented function

In [30]:
def test(hashtable_func):
    inputs = [0xa582041, 0xa592041, 0xa5a2041, 0xa582042, 0xa592042, 0xa5a2042, 0xa582043, 0xa592043]
    expected_outputs = [4, 8, 3, 1, 5, 9, 7, 2]
    for inp, expected in zip(inputs, expected_outputs):
        assert hashtable_func(inp) == expected
    print("test successful")
        
        
test(hashtable)

test successful


## Attempting to compress the hash table (so it requires less space)
Because currently 3 places in the array are unused

In [27]:
for i in range(8, 30):
    hash_func = lambda x: x % i
    if not isPerfectHashfunction(hash_func, inputs):
        continue
    lut = buildLookupTable(hash_func, inputs, outputs)
    print(i, len(lut), lut)

12 12 [0, 8, 5, 2, 0, 3, 9, 0, 0, 4, 1, 7]
13 11 [0, 0, 0, 4, 1, 7, 8, 5, 2, 3, 9]
19 19 [9, 0, 0, 0, 0, 0, 0, 0, 4, 1, 7, 0, 0, 8, 5, 2, 0, 0, 3]
20 19 [0, 8, 5, 2, 0, 4, 1, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 9]
21 18 [0, 0, 0, 0, 0, 3, 9, 0, 0, 0, 8, 5, 2, 0, 0, 4, 1, 7]
23 23 [9, 0, 0, 0, 4, 1, 7, 0, 0, 0, 0, 0, 0, 8, 5, 2, 0, 0, 0, 0, 0, 0, 3]
24 19 [0, 8, 5, 2, 0, 0, 0, 0, 0, 4, 1, 7, 0, 0, 0, 0, 0, 3, 9]
25 24 [0, 0, 0, 0, 0, 0, 0, 3, 9, 0, 4, 1, 7, 0, 0, 0, 0, 0, 0, 0, 0, 8, 5, 2]
26 22 [0, 0, 0, 4, 1, 7, 0, 0, 0, 3, 9, 0, 0, 0, 0, 0, 0, 0, 0, 8, 5, 2]
27 25 [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 7, 0, 0, 0, 0, 8, 5, 2, 0, 0, 0, 0, 3, 9]
28 20 [0, 4, 1, 7, 0, 3, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 5, 2]
29 17 [0, 0, 0, 0, 0, 0, 3, 9, 0, 0, 8, 5, 2, 0, 4, 1, 7]


Result: mod 13 produces also 3 empty places but they are all in the beginning, so they can be eliminated by subtracting 3

Build lookup table

In [36]:
lut = buildLookupTable(lambda x: x % 13 - 3, inputs, outputs)
lut

[4, 1, 7, 8, 5, 2, 3, 9]

Create the function

In [37]:
def hashtableCompressed(entry):
    return lut[entry % 13 - 3]

Test the implemented function

In [38]:
test(hashtableCompressed)

test successful


Measure time

In [41]:
start = default_timer()
for _ in range(1_000_000):
    hashtable(0xa582041) == 4
default_timer() - start

0.12916204001521692